In [ ]:
import numpy as np
import pandas as pd
pd.set_option('mode.use_inf_as_na', True)

from leaspy import AlgorithmSettings, __watermark__

from my_utils import compute_dcm_model
# Watermark trace with all important packages versions
__watermark__

In [ ]:
## Create DataFrame with random data
nSubjects = 100
nBiomarkers = 2 
nVisits = 6
labels = np.random.choice([0, 1], size=nSubjects, replace=True)

idx_subj = [f'sub_{i+1}' for i in range(nSubjects)]

#
markers = [f'Marker_{j+1}' for j in range(nBiomarkers)]
biomarker_values = []
age_list = []
# Generate random ascending data for biomarkers (between 0 and 1)
for i in range(nSubjects):
    biomarker_list = []
    age = np.random.choice(np.arange(50, 90, 1), size=nVisits, replace=False)
    for j in range(nBiomarkers):
        biom_values = np.random.choice(np.arange(0, 1, 0.01), size=nVisits, replace=False)
        biomarker_list.append(np.sort(biom_values))
    biomarker_values.append(np.array(biomarker_list).T)
    #age_list.append(age)
 
    idx = pd.MultiIndex.from_product([idx_subj, np.sort(age)], names=['ID', 'TIME'])

df_example = pd.DataFrame((np.array(biomarker_values).reshape(nSubjects * nVisits, nBiomarkers)), index=idx, columns=markers)

In [ ]:
# Model parameters

nFold = 5 # number of folds for cross-validation
n_timepoint=90
age_min = 40
age_max = 100
timepoints = np.linspace(age_min, age_max, n_timepoint)

model_type = "logistic"
nb_source = 1 # number of features - 1. 
algo_settings = AlgorithmSettings('mcmc_saem', n_iter=1000, seed=0)  # Set FITTING parameters      
algo_settings.set_logs(path='logs', console_print_periodicity=None,  overwrite_logs_folder=True)
settings_personalization = AlgorithmSettings('scipy_minimize', seed=0) # Set PERSONALIZATION parameters

In [ ]:
SubList = df_example.index.get_level_values('ID').unique().tolist()
model_lst, ip_df, AverageTraj, mae, r2 = compute_dcm_model(df_example, SubList, labels, markers, model_type, nb_source, algo_settings, settings_personalization, 
                      timepoints, K=5)